In [ ]:
import os

current_dir = os.getcwd()

# TWO FOLDERS UP
data_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir, 'data'))
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')

train_dirs = []
for i in range(1, 5):
    train_dirs.append(os.path.join(train_dir, 'train' + str(i)))

validation_dir = os.path.join(data_dir, 'train', 'train5')

print(current_dir)
print(data_dir)
print(test_dir)
print(train_dir)
print(validation_dir)

In [ ]:

from keras.utils import image_dataset_from_directory
import tensorflow as tf

# Load training datasets from train1 to train4
train_datasets = []
IMG_SIZE = 150
BATCH_SIZE = 32

for i in range(1, 5):
    dataset = image_dataset_from_directory(train_dirs[i-1], image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')
    train_datasets.append(dataset)
 
train_dataset = train_datasets[0]
for dataset in train_datasets[1:]:
    train_dataset = train_dataset.concatenate(dataset)

validation_dataset = image_dataset_from_directory(validation_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')


test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, label_mode='categorical')

class_names = validation_dataset.class_names
class_names = [class_name.split('_')[-1] for class_name in class_names]

print(class_names)

In [ ]:
import matplotlib.pyplot as plt

for data, _ in train_dataset.take(1):
    for i in range(9):
        plt.imshow(data[i].numpy().astype('uint8'))
        plt.show()
    break

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

In [ ]:
# from tensorflow import keras
# import numpy as np

# def get_features_and_labels(dataset):
#     all_features = []
#     all_labels = []
#     for images, labels in dataset:
#         preprocessed_images = keras.applications.vgg16.preprocess_input(images)
#         features = base_model.predict(preprocessed_images)
#         all_features.append(features)
#         all_labels.append(labels)
#     return np.concatenate(all_features), np.concatenate(all_labels)


# validation_features, validation_labels = get_features_and_labels(validation_dataset)
# np.save('validation_features.npy', validation_features)
# np.save('validation_labels.npy', validation_labels)
# validation_features = None
# validation_labels = None
# print("Validation features and labels saved")

# test_features, test_labels = get_features_and_labels(test_dataset)
# np.save('test_features.npy', test_features)
# np.save('test_labels.npy', test_labels)
# test_features = None
# test_labels = None
# print("Test features and labels saved")

# train_features, train_labels = get_features_and_labels(train_dataset)
# np.save('train_features.npy', train_features)
# np.save('train_labels.npy', train_labels)
# train_features = None
# train_labels = None
# print("Train features and labels saved")

In [ ]:
from numpy import load

train_features = load('features_T/train_features.npy')
train_labels = load('features_T/train_labels.npy')

validation_features = load('features_T/validation_features.npy')
validation_labels = load('features_T/validation_labels.npy')

test_features = load('features_T/test_features.npy')
test_labels = load('features_T/test_labels.npy')

**Architecture:**



In [ ]:
from tensorflow import keras
from keras.applications import VGG16
from keras import layers

inputs = keras.Input(shape=(4, 4, 512))

x = layers.Flatten()(inputs)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(10, activation="softmax")(x)  # Softmax for multi-class classification

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_acc', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=1e-6)

early_stop = EarlyStopping(monitor='val_acc', 
                           patience=5,
                           restore_best_weights=True)
model_checkpoint = ModelCheckpoint('models/T01/checkpoints/T01-cp.h5', save_best_only=True)

history = model.fit(
    train_features, train_labels,
    epochs=50,
    validation_data=(validation_features, validation_labels),
    callbacks=[early_stop, model_checkpoint, learning_rate_reduction])

In [ ]:
keras.models.save_model(model, 'models/T01/T01-model.h5')

In [ ]:
keras.models.load_model('models/T01/T01-model.h5')

In [ ]:
val_loss, val_acc = model.evaluate(validation_features, validation_labels)
print('val_acc:', val_acc)

In [ ]:
import matplotlib.pyplot as plt

# Extract the history from the training process
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Plot the training and validation accuracy
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

# Plot the training and validation loss
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

y_pred = np.argmax(model.predict(validation_features), axis=1)
y_true = np.argmax(validation_labels, axis=1)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np

# Load an image
img_path = train_dirs[0] + '/006_frog/alytes_obstetricans_s_000179.png'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150), interpolation='bilinear')

# Preprocess the image for VGG16
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

plt.imshow(img)
plt.show()

print(img_array.shape)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Extract features using VGG16
features = base_model.predict(img_array)

flattened_features = features.reshape((features.shape[0], -1))

# Predict using your custom model
result = model.predict(flattened_features)

print("Result: ", result.round())
print("Predicted class: ", class_names[np.argmax(result)])
print("True class: ", img_path.split('/')[-2].split('_')[-1])
